In [1]:
# import of important libraries
from datetime import datetime
from datetime import timedelta
import random
import re
from functions import load_csv, write_csv, sort
import csv

In [2]:
# Before describing all the classes, one can take a look in the functions.py file which also uses some self-built functions that will be used throughout the program.

# PART 1: CLASSES

#### Appointment

In [3]:
# starting with appointment_sched class, it has as attributes a list of appointment's where appointment itself is a class within the appointment_sched class which has as attributes a type which is a date in the conext here, a health professional member and a patient which is a list of information. When a new appoitnment instance is called, it is possible to use also some methods. Add_appoint adds an appointment to the appointment list, del_appoint deletes an appointment from the appointment list and find_appoint finds the next suitable appointment.
class appointment_sched:
  
    list1 = []
    
    class appointment:

        def __init__(self, type, health_pro,patient):
            self.type = type
            self.health_pro = health_pro
            self.patient = patient

        # add_appoint simply adds a new appointment instance (gathered into a list) to the list of all appointments (No check).
        def add_appoint(self):
            appointment_sched.list1.append([self.type,self.health_pro,self.patient[0],self.patient[1],self.patient[2]])

        # the del_appoint method checks whether the mentioned type, staff and patient is in the appointment_sched list called "list1". If so, then it saves the index in a variable "index" and the counter becomes 1. If the counter is at the end still 0, then a message will be shown to indicate that no appointment has been found. Otherwise, the appointment will be removed from the schedule list. 
        def del_appoint(self):
            counter = 0
            index = 0
            for i in range(len(appointment_sched.list1)):
                if appointment_sched.list1[i][0] == str(self.type) and appointment_sched.list1[i][1] == self.health_pro and appointment_sched.list1[i][2] == self.patient[0]:
                    index = i
                    counter += 1
            if counter == 0:
                print("Appointment not found. Please restart again. ")
            if counter == 1:
                appointment_sched.list1.pop(index)

        # the find_appoint method works with the datetime library and simply tries to find a suitable date that fits in the doctors schedule. It mainly differentiates between 3 cases. Appointment wishes after 18 o' clock should be delayed to the next day (08:00), the same is true for the lunch break (12:30 should be delayed to 13:30) and all other blocked dates (where is already a meeting for the doctor) should lead to proposal for the next meeting (00:30 later). This whole processes will be done until a free date is found (see the while loop which only terminates when the counter is 1 (i.e. if a date is found)). 
        def find_appoint(self):
            counter = 0
            while counter == 0:
                for i in range(0,len(appointment_sched.list1)):
                    if self.type.hour == 18:
                        self.type = self.type + timedelta(days = 1, hours = -10)
                        continue
                    if self.type.hour == 12 and self.type.minute == 30:
                        self.type = self.type + timedelta(hours = 1)
                        continue
                    if appointment_sched.list1[i][0] == str(self.type) and appointment_sched.list1[i][1] == self.health_pro:
                        n = 30
                        self.type = self.type + timedelta(minutes=n)
                        continue
                counter = 1
            return self.type

#### Patient

In [4]:
# the patient class has three attributes (name,address,phone) and these attributes become powerful in using one of the two methods. request_repeat is a method that helps to show a doctor the previous prescription of a patient so that he can decide whether to continue or not with one of them. request_appoint is directly connected to the make_appoint method in the receptionist class which at the end enables to call the _add_appoint or del_appoint method of the appointment_sched class so that meetings can be added or removed from the schedule list. 
class patient:

    def __init__(self, name, address, phone):

        self.name = name
        self.address = address
        self.phone = phone

    # this method takes an existing prescription list and an employee number as arguments and checks for the mentioned patient and the corresponding doctor whether there are some previous prescription in the list. If so, the issue_presc of the doc class is called.
    def request_repeat(self,list_p,e_number):    
        counter = 0
        tmp = []
        for i in range(0,len(list_p)):
            if list_p[i][1] == self.name and list_p[i][2] == self.address and list_p[i][3] == self.phone and list_p[i][4] == e_number:
                tmp.append(list_p[i])
                counter += 1
        if counter == 0:
            print('Please restart again. No prescription found. ')
        else: 
            doc.issue_presc(tmp)

    # this method differentiates between two cases which depend on the value of the given variable "tmp". If this variable is "1" then it calls the make_appoint method of the receptionist class. If this variable is equal to "2" then it calls cancel_appoint of the receptionist class.
    def request_appoint(self,tmp,list_health,list_appoint):        
        for i in list_health:
            print(i)
        staff = input('Above you can see all the doctors that work in our practice. Please search for your doctor and enter the number that appears to the most left. : ')
        if tmp == '1':
            receptionist.make_appoint(list_appoint,staff,[self.name,self.address,self.phone])
        if tmp == '2':
            receptionist.cancel_appoint(list_appoint,staff,[self.name,self.address,self.phone])

#### Health Professional

In [5]:
# this class is used in the health_pro class later and it essentially returns the key and value section of a dictionary.
class IterMixin(object):
    def __iter__(self):
        for attr, value in self.__dict__.items():
            yield attr, value

# health pro class consists of 2 attributes (name, employee number). The consultation method will not be used in the program here but it could be interesting for future purpose. It essentially takes a summary text of a consultation and then searches for one key word, which should be one included in the list of possible consultations seen above. If it finds one, then it returns it. 
class health_pro(IterMixin):

    def __init__(self, name, employ_number):
        self._name = name
        self._employ_number = employ_number

    def consultation(text):
        list = re.sub("[^\w\s]", "", text).split()
        print(text)
        for i in possible_consultations:
            for j in list:
                if i == j:
                    return i
                
    # the properties below restrict a health_pro in its abilities to work with the attributes. In other words, a health_pro instance should only be able to be read and not to be changed (no setter option).                   
    @property
    def name(self):
        return self._name
        
    @property
    def number(self):
        return self._employ_number


#### Doctor and Nurse

In [6]:
# the doc class stands for a doctor and it inherites all the attributes and methods of the health_pro class.
class doc(health_pro):

    def __init__(self, name, employ_number):
        super().__init__(name,employ_number)
    
    # Beside all the methods of the health_pro class, a doctor class also has this authorization method which simply allows to check if the mentioned employee number is a doctor and therefore he is authorized to issue repeated prescriptions.
    def authorization(self,list):
        for i in list:
            if i[0] == self.number and i[2] == "doc":
                return True
        return False

    # this methods takes a list as argument and this list will be shown to the doctor (including an index number) so that he can decide which one he wants to choose and hence to issue to the patient. 
    def issue_presc(list):
        for i in range(0,len(list)):
            list[i] = [i] + list[i]
        print(list)
        issue = input('Which one do you want to issue to the patient? Please enter the number at the most left of the prescription: ')
        for i in range(0, len(list)):
            if str(list[i][0]) == issue:
                return print(list[i])          

# the nurse class inherites all the attributes and methods of the health_pro class but it has no own attribute + methods.
class nurse(health_pro):
    def __init__(self, name, employ_number, docORnurse,count):
        super().__init__(name,employ_number,docORnurse,count)

#### Prescription

In [7]:
# the prescription class has five attributes: a name for the prescription ("type"), a patient tuple, a doctor (only employee number), a quantity and a dosage of the prescription. There are no methods within the class and it will therefore never be used throughout the main program. Nevertheless, futur purpose easily allow to reuse and adapte the existing class of "prescription". 
class prescription:

    def __init__(self, type, patient, doc, quantity, dosage):
        self.type = type
        self.patient = patient
        self.doc = doc
        self.quantity = quantity
        self.dosage = dosage    

#### Receptionist

In [8]:
# the receptionist class contains as attributes the employee name and his number and it consists of two main methods: make _appoint and cancel_appoint. Beside these two methods, there are also properties (only read option) and two further methods whereas authorization is similar to the authorization method in the doctor class (only that now a receptionist sould be checked) and the find_a_doc method allows to check for available doctors that can still take new patients. 
class receptionist:

    def __init__(self, name, employ_number):
        self._name = name
        self._employ_number = employ_number

    # As already seen in the health_pro class, here also a receptionist instance should only be readable and it should not possible to change. (Static csv files)
    @property
    def name(self):
        return self._name
        
    @property
    def number(self):
        return self._employ_number

    # Similar to auhtorization in doc class
    def authorization(self,list):
        for i in list:
            if i[0] == self.number and i[2] == "receptionist":
                return True
        return False
    
    # The idea here is to check which doctor still has availabilities with respect to new patients. Therefore it needs a patient and a healt_pro list and it loops over all existing patients and looks up which health_pro they have and counts them. After this process, it checks the health_pro list for doctors/nurse which still have less than 500 patients and returns the first one that it finds (i.e. new patients have no choice, they get the first doctor who turns out to be available).
    def find_a_doc(pat_file,length_pat,health_file,length_health,reference_number):
        tmp = []
        for i in load_csv(health_file,length_health):
            i.append(0)
            tmp.append(i)
        for i in load_csv(pat_file,length_pat):
            for j in range(0,len(tmp)):
                if j+1 == int(i[-1]):
                    tmp[j][-1] += 1
        for i in range(0,len(tmp)):
            if int(tmp[i][-1]) < int(reference_number) and tmp[i][2] == "doc":
                return tmp[i][0]

    # this method always starts in finding a suitable appointment. For this it asks the patient which is his preferred date and time. After this, the date1 variable does some concatenation and then transforms it into a datetime format. At the end it starts the request whether the mentioned date is available. If not, the find_appoint function of the appointment_sched class will return the next available date for the mentioned doctor. The final step in make_appoint is then to ask the patient if he agrees with the proposed date or not. If not, the while loop continues. Otherwise, the while loop will be left and the receptionist did his work and the appointment can finally be added to the schedule.     
    def make_appoint(list,health_pro,patient):
        question = "2"
        while question == "2":
            tmp1d = input('Please enter the date (m/d/y) of your preferred appointment : ')
            tmp1t = input('Please enter the time (h:m) of your preferred appointment : ') 
            date1 = datetime.strptime(tmp1d + ' ' + tmp1t + ':' + '00.000', '%m/%d/%Y %H:%M:%S.%f')
            date1 = appointment_sched.appointment(date1,health_pro,patient).find_appoint()
            print(date1)
            question = input("The next available appointment is the one seen above. Do you agree (enter 1) or do you want that we continue to search (enter 2)? ") 
        appointment_sched.appointment(date1,health_pro,patient).add_appoint()

    # cancel_appoint works similar but it searches for the appointment that the patient mentiones. It simply sends (after the date transformation) a del_appoint request and this request checks whether an entry was found or not. If not, a failure message will be shown to the receptionist.    
    def cancel_appoint(list,health_pro,patient):
        tmp1d = input('Please enter the date (m/d/y) of your appointment : ')
        tmp1t = input('Please enter the time (h:m) of your appointment : ')  
        date1 = datetime.strptime(tmp1d + ' ' + tmp1t + ':' + '00.000', '%m/%d/%Y %H:%M:%S.%f')
        appointment_sched.appointment(date1,health_pro,patient).del_appoint()

# PART 2: MAIN

#### Connection to appointment file (csv)

In [ ]:
# It begins in reading the appointment schedule data file that is important to run the app. To read the file, the csv library is used and and the self-built function load_csv which you can find "functions". 

# load the appointment file and use the appointment_sched.list1 to save that file in a list which is already inside of a class.
appointment_sched.list1 = load_csv('appoint_testdata1.csv',5)

#### Inclusion of the user

In [ ]:
# the main program uses a never ending while loop so that the user will always be integrated (keepgoing is always "True")
keepgoing = True
while keepgoing == True:
    
    # the idea of the app is that a patient is on the phone line with a receptionist or a doctor and the receptionist or doctor itself uses the app.
    # now, the application asks whether the use puropse of the app is to have a new appointment or only to ask for a repetitive prescription
    tmp = input('When you call for an appointment, please enter 1. When you call for the repetition of your prescription, please enter 2: ')
    
    # After this the health professional has to enter his own personal and professional data. 
    e_name1 = input("You must be an employee of our surgery. Please enter your given name: ").lower()
    e_name2 = input("You must be an employee of our surgery. Please enter your last name: ").lower()
    e_name = e_name1 + '_' + e_name2
    e_number = input("You must be an employee of our surgery. Please enter your employee number: ").lower()   
    
    # when the purpose of the call is to look for an appointment, then only a receptionist should be allowed to use the app. 
    if tmp == "1":
        
        # Hence, the authorization method of the receptionist class is called in using the current receptionist instance
        if receptionist(e_name,e_number).authorization(load_csv('health_pro.csv',3)) == False:
            continue
        
        # the app continues in registering the key information of a patient, name, address and phone number and whether the patient is new or an existing one.
        name1 = input('Please enter the given name of your patient: ').lower()
        name2 = input('Please enter the last name of your patient: ').lower()
        name = name1 + '_' + name2
        address1 = input('Please enter the street of the address of your patient: ').lower() 
        address2 = input('Please enter the number of the street of your patient: ').lower()
        address3 = input('Please enter the post code of the address of your patient: ').lower()
        address4 = input('Please enter the city name of the address of your patient: ').lower()
        address = address1 + '_' + address2 + '_' + address3 + '_' + address4
        phone = input('Please enter the phone number of your patient: ').lower()
        new = input('If your patient is a new one, please enter 1 and otherwise 0: ')
        
        # if the patient mentiones that he is a new one, then the program has to check which doctor can take this new patient in his portfolio. Therefore, the tmp list (a copy of all the data in the health_pro csv file) also add a count variable to every entry which is at the beginning 0. Aftter that, it uses the find_a_doc method of the receptionist class which then makes the check which health_pro can take that new patient. 
        # If the patient is not new, then the program don't has to look up for a doctor and can already ask the next question
        if new == "1":
            tmp = []
            for i in load_csv('health_pro.csv',3):
                i.append(0)
                tmp.append(i)
            write_csv('pat_testdata1.csv',"add",[[name,address,phone,receptionist.find_a_doc('pat_testdata1.csv',4,'health_pro.csv',3,500)]])
            
            # Because the number of the doctor will be needed later on, it is important to show this number to the receptionist.
            print(f"The number of your doctor is {receptionist.find_a_doc('pat_testdata1.csv',4,'health_pro.csv',3,500)}. Please remeber this for further steps") 
        
        # after registering the personal data of the patient, the receptionist asks whether the patient want to cancel or make a new appointment.
        tmp1 = input('Do you want to cancel (enter 2) or make (enter 1) a new appointment? ').lower()
        
        # with all these information (personal data patient, responsible doctor and a variable tmp1 which indicates whether "make" or "cancel" is used), a patient instance is created and the request_appont method is directly called.
        patient(name,address,phone).request_appoint(tmp1,load_csv('health_pro.csv',3),appointment_sched.list1)
        
        # depending on the value of tmp1, the program has to differentiate the writing process into the appointment schedule csv file. If a new appointment has finally been added, the append modus is once again called. On the other side, if tmp1 == 2 (cancel an appointment) is transmitted, the overwriting process restarts again (first row in "w" modus and "a" modus for the following ones).
        if tmp1 == "1":
            list_tmp = []
            for i in appointment_sched.list1[-1]:    
                list_tmp.append(i)
            write_csv('appoint_testdata1.csv',"add",[list_tmp])

        if tmp1 == "2":
            write_csv('appoint_testdata1.csv',"delete",appointment_sched.list1)
       
    # going back to what tmp should represent, a value of 2 means that the patient calls for repetitive prescription
    if tmp == '2':
        
        # make first the check whether the employee using the app is authorized to make an appointment, in other words, he/she is a doctor. (authorization method of the doc class is called)
        if doc(e_name,e_number).authorization(load_csv('health_pro.csv',3)) == False:
            continue
        
        # the call starts with registering the key information of a patient, name, address and phone number and whether the patient is new or an existing one.
        name1 = input('Please enter the given name of your patient: ').lower()
        name2 = input('Please enter the last name of your patient: ').lower()
        name = name1 + '_' + name2
        address1 = input('Please enter the street of the address of your patient: ').lower() 
        address2 = input('Please enter the number of the street of your patient: ').lower()
        address3 = input('Please enter the post code of the address of your patient: ').lower()
        address4 = input('Please enter the city name of the address of your patient: ').lower()
        address = address1 + '_' + address2 + '_' + address3 + '_' + address4
        phone = input('Please enter the phone number of your patient: ').lower()
        
        # finally the request_repeat method of the patient class will be called while creating a new patient instance.
        patient(name,address,phone).request_repeat(load_csv('presc_testdata1.csv',7),e_number)

    # never ending program => continue    
    continue

When you call for an appointment, please enter 1. When you call for the repetition of your prescription, please enter 2: 1
You must be an employee of our surgery. Please enter your given name: hans
You must be an employee of our surgery. Please enter your last name: haas
You must be an employee of our surgery. Please enter your employee number: weger
When you call for an appointment, please enter 1. When you call for the repetition of your prescription, please enter 2: 12
You must be an employee of our surgery. Please enter your given name: g
You must be an employee of our surgery. Please enter your last name: g
You must be an employee of our surgery. Please enter your employee number: g
When you call for an appointment, please enter 1. When you call for the repetition of your prescription, please enter 2: 1
You must be an employee of our surgery. Please enter your given name: ingrid
You must be an employee of our surgery. Please enter your last name: durby
You must be an employee of o